In [ ]:
import threading
import time
import queue
import random

class VegetableMarket:
    def __init__(self, capacity):
        self.capacity = capacity
        self.market_queue = queue.Queue(maxsize=capacity)
        self.farmers = []
        self.consumers = []
        self.lock = threading.Lock()

    def add_farmer(self, farmer):
        self.farmers.append(farmer)

    def add_consumer(self, consumer):
        self.consumers.append(consumer)

    def farmer_produce_vegetable(self, farmer_id, vegetable_type):
        while True:
            with self.lock:
                if self.market_queue.full():
                    print(f"Farmer {farmer_id} waiting - Market is full.")
                else:
                    print(f"Farmer {farmer_id} produced {vegetable_type}.")
                    self.market_queue.put(vegetable_type)
            time.sleep(random.uniform(1, 3))

    def consumer_buy_vegetable(self, consumer_id, vegetable_type):
        while True:
            with self.lock:
                if vegetable_type not in self.market_queue.queue:
                    print(f"Consumer {consumer_id} waiting for {vegetable_type}.")
                else:
                    print(f"Consumer {consumer_id} bought {vegetable_type}.")
                    self.market_queue.get()
            time.sleep(random.uniform(2, 5))

def simulate_market():
    market = VegetableMarket(capacity=5)

    for i in range(3):
        farmer = threading.Thread(target=market.farmer_produce_vegetable, args=(i + 1, f"Vegetable-{i + 1}"))
        market.add_farmer(farmer)
        farmer.start()

    for i in range(5):
        consumer = threading.Thread(target=market.consumer_buy_vegetable, args=(i + 1, f"Vegetable-{i + 1}"))
        market.add_consumer(consumer)
        consumer.start()

    for farmer in market.farmers:
        farmer.join()

    for consumer in market.consumers:
        consumer.join()

if __name__ == "__main__":
    simulate_market()

